In [1]:
import easyocr
import re
import cv2
import numpy as np

In [2]:
reader = easyocr.Reader(['en'])  # с ru хуже

In [3]:
def adjust_gamma(image, gamma=1.2):
    # build a lookup table mapping the pixel values [0, 255] to
    # their adjusted gamma values
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")

    # apply gamma correction using the lookup table
    return cv2.LUT(image, table)

# Automatic brightness and contrast optimization with optional histogram clipping
def automatic_brightness_and_contrast(image, clip_hist_percent=1):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Calculate grayscale histogram
    hist = cv2.calcHist([gray],[0],None,[256],[0,256])
    hist_size = len(hist)
    
    # Calculate cumulative distribution from the histogram
    accumulator = []
    accumulator.append(float(hist[0]))
    for index in range(1, hist_size):
        accumulator.append(accumulator[index -1] + float(hist[index]))
    
    # Locate points to clip
    maximum = accumulator[-1]
    clip_hist_percent *= (maximum/100.0)
    clip_hist_percent /= 2.0
    
    # Locate left cut
    minimum_gray = 0
    while accumulator[minimum_gray] < clip_hist_percent:
        minimum_gray += 1
    
    # Locate right cut
    maximum_gray = hist_size -1
    while accumulator[maximum_gray] >= (maximum - clip_hist_percent):
        maximum_gray -= 1
    
    # Calculate alpha and beta values
    alpha = 255 / (maximum_gray - minimum_gray)
    beta = -minimum_gray * alpha  
    auto_result = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    return (auto_result, alpha, beta)

def clahe(frame):

    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    lab[:,:,0] = clahe.apply(lab[:,:,0])

    # Converting image from LAB Color model to BGR color space
    image_clahe = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    return image_clahe

In [4]:
def image_preparation(frame):

    # minmal size rectangle, px   
    frame_for_auto = frame
    frame_for_auto = cv2.fastNlMeansDenoisingColored(frame_for_auto ,None, 2, 2, 4, 4)
    frame_for_auto = clahe(frame_for_auto)
    frame_for_auto = adjust_gamma(frame_for_auto)        
    
    frame_auto, alpha, betta = automatic_brightness_and_contrast(frame_for_auto, clip_hist_percent=1)

    out_frame = frame_auto
    return out_frame

def get_rects(frame):
    minSize = int(frame.shape[0]/2.1)
    return reader.readtext(frame, min_size = minSize, width_ths=0.1)

In [8]:
def run_processing(filename, out_filename='video'):
    cap = cv2.VideoCapture(filename)

    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))

    # fourcc = cv2.VideoWriter_fourcc(*'X264')    
    # out=cv2.VideoWriter(f'{out_filename}.mkv',
    #                     fourcc,
    #                     24, (frame_width, frame_height), True)

    out=cv2.VideoWriter(f'{out_filename}.avi',
                    cv2.VideoWriter_fourcc('D','I','V','X'),
                    24, (frame_width, frame_height), True)

    # ind = 0
    # errInd = 49

    if not cap.isOpened():
        print("Error opening video file")
    else:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            frame_for_rect = frame
            frame_for_rect = image_preparation(frame)
            rects = get_rects(frame_for_rect) # OCR function

            # frame = frame_for_rect
           
            if len(rects) > 0:
                for elem in rects:
                    number_str = re.sub("[^0-9]", "", elem[1].replace(" ", ""))                    
                    if len(number_str) == 0:
                        continue

                    # если символов меньше или больше 8, то нужно добавлять в датасет для обучения
                    # дополнительно можно проверять верность полученного номера (по принципу его формирования)                    

                    frame = cv2.rectangle(frame, np.array(elem[0][0]).astype(int), np.array(elem[0][2]).astype(int), (0, 0, 255), 2, )
                    frame = cv2.putText(frame, number_str,
                                        np.array(elem[0][0]).astype(int), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1.2,
                                        color=(0, 0, 255), thickness=2)

            # ind += 1

            cv2.imshow('Vagons', frame)
            out.write(frame)
            key = cv2.waitKey(1)
            if key == 27: # Esc нажать для остановки видео
                break
    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [11]:
# run_processing('Sample/vagons.mp4', out_filename='processed_cisterns')
run_processing('Sample/cisterns.avi', out_filename='processed_cisterns1')